In [1]:
import os
os.environ["LOGLEVEL"] = "PROD"

import numpy as np
import pandas as pd
from neuro_dmt.models.bluebrain.circuit.model import BlueBrainCircuitModel
from neuro_dmt.models.bluebrain.circuit.mock.test.mock_circuit_light\
    import circuit_composition, circuit_connectivity
from neuro_dmt.models.bluebrain.circuit.mock.circuit import\
    MockCircuit

('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mtype', 'etype', 'synapse_class')
('region', 'layer', 'depth', 'height', 'mtype', 'etype', 'synapse_class')


In [2]:
mock_circuit =\
    MockCircuit.build(
        circuit_composition,
        circuit_connectivity)
circuit_model =\
    BlueBrainCircuitModel(
        mock_circuit,
        label="BlueBrainCircuitModelMock")
conn_prob = circuit_model.connection_probability

In [3]:
pre_synaptic_cells =\
    circuit_model.cells\
                 .sample(n=100)\
                 .rename(columns=conn_prob._at("pre_synaptic"))
post_synaptic_cells =\
    circuit_model.cells\
                 .sample(n=100)\
                 .rename(columns=conn_prob._at("post_synaptic"))
conn_prob.get_summary(
    pre_synaptic_cell_group=pre_synaptic_cells,
    post_synaptic_cell_group=post_synaptic_cells)

pairs_total               10000.0000
pairs_connected             209.0000
connection_probability        0.0209
Name: 0, dtype: float64

In [4]:
cp_pairs_generator = conn_prob.get_pairs(
    pre_synaptic_cells,
    post_synaptic_cells,
    upper_bound_soma_distance=1000)
cp_pairs = list(
    cp_pairs_generator)
cp_pairs[0].columns

Index(['pre_synaptic_region', 'pre_synaptic_layer', 'pre_synaptic_nucleus',
       'pre_synaptic_mtype', 'pre_synaptic_etype', 'pre_synaptic_morph_class',
       'pre_synaptic_synapse_class', 'x', 'y', 'z', 'pairs',
       'post_synaptic_region', 'post_synaptic_layer', 'post_synaptic_nucleus',
       'post_synaptic_mtype', 'post_synaptic_etype',
       'post_synaptic_morph_class', 'post_synaptic_synapse_class', 'x', 'y',
       'z'],
      dtype='object')

In [5]:
np.isnan(cp_pairs[0].pairs.values).sum()

0

In [9]:
pre_synaptic_cell_group = circuit_model.cells.sample(n=100)
post_synaptic_cell_group = circuit_model.cells.sample(n=100)
conn_prob(
    pre_synaptic_cell_group=pre_synaptic_cell_group,
    post_synaptic_cell_group=post_synaptic_cell_group,
    upper_bound_soma_distance=1000,
    with_summary_statistics=True)

pairs_total               8918.000000
pairs_connected            168.000000
connection_probability       0.018838
Name: 0, dtype: float64

In [10]:
conn_prob(
    pre_synaptic_cell_group=pre_synaptic_cell_group,
    post_synaptic_cell_group=post_synaptic_cell_group,
    upper_bound_soma_distance=100.,
    with_summary_statistics=True)

pairs_total               356.000000
pairs_connected             5.000000
connection_probability      0.014045
Name: 0, dtype: float64

In [12]:
conn_prob(
    pre_synaptic_cell_group=pre_synaptic_cell_group,
    post_synaptic_cell_group=post_synaptic_cell_group,
    with_summary_statistics=True)

pairs_total               10000.0000
pairs_connected             189.0000
connection_probability        0.0189
Name: 0, dtype: float64

In [4]:
from dmt.tk.parameters import Parameters
from neuro_dmt.models.bluebrain.circuit.adapter import\
    BlueBrainCircuitAdapter

parameters_pathways_with_soma_distance = Parameters(
    circuit_model.pathways(
        {"mtype",}
    ).assign(
        upper_bound_soma_distance=500.
    ))
adapter = BlueBrainCircuitAdapter()
parameters_for_sampling =\
    parameters_pathways_with_soma_distance.for_sampling(size=10)
adapter.get_connection_probability(
    circuit_model,
    **parameters_for_sampling[0])

('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')


dummy_variable
0    0.02102
Name: connection_probability, dtype: float64

In [5]:
parameters_pathways_with_soma_distance = Parameters(
    circuit_model.pathways(
        {"mtype",}
    ).assign(
        upper_bound_soma_distance=50.
    ))
adapter = BlueBrainCircuitAdapter()
parameters_for_sampling =\
    parameters_pathways_with_soma_distance.for_sampling(size=10)
adapter.get_connection_probability(
    circuit_model,
    **parameters_for_sampling[0])

dummy_variable
0    0.029801
Name: connection_probability, dtype: float64

In [6]:
from neuro_dmt.models.bluebrain.circuit.model.pathway import\
    ConnectionProbabilityBySomaDistance
conn_prob_sd = ConnectionProbabilityBySomaDistance(circuit_model=circuit_model)

In [ ]:
conn_prob_sd(
    pre_synaptic_cell_group=pre_synaptic_cells,
    post_synaptic_cell_group=post_synaptic_cells)

In [11]:
conn_prob(
    pre_synaptic_cell_group=pre_synaptic_cells,
    post_synaptic_cell_group=post_synaptic_cells,
    with_summary_statistics=True).loc[0]["connection_probability"]

0.0199

In [7]:
measured_pairs = conn_prob.get_pairs(
    pre_synaptic_cells=pre_synaptic_cells,
    post_synaptic_cells=post_synaptic_cells)

In [8]:
cpdf =\
    pd.concat([
        pairs[
            ["pairs"]
        ].reset_index(
            drop=True
        ).assign(
            group=0
        ).groupby(
            ["group"]
        ).agg(
            conn_prob.aggregators
        ).pairs.rename(
            columns=conn_prob.columns
        ) for pairs in measured_pairs]
    ).groupby(
        ["group"]
    ).agg(
        "sum"
    ).assign(**{
        conn_prob.phenomenon: conn_prob.definition}
    )

In [10]:
cpdf.loc[0]

pairs_total               10000.0000
pairs_connected             199.0000
connection_probability        0.0199
Name: 0, dtype: float64

In [ ]:
pre_synaptic_group_columns = ["pre_synaptic_mtype"]
post_synaptic_group_columns = ["post_synaptic_mtype"]
result = pairs_df[
    ["pairs", "soma_distance"] + pre_synaptic_group_columns + post_synaptic_group_columns
].groupby(
    pre_synaptic_group_columns
    + post_synaptic_group_columns
    + ["soma_distance"]
).agg(["size", "sum"])
result.head()

In [ ]:
result = pairs_df[
    ["pairs"]
].assign(
    group=0
).groupby(
    ["group"]
).agg(
    ["sum", "size"]
).pairs.rename(
    columns=conn_prob.columns
)
conn_prob.definition(result.loc[0])

In [ ]:
result = pairs_df[
    ["soma_distance"] + ["pairs"]
].assign(
    group=0
).groupby(
    ["group", "soma_distance"]
).agg(
    ["sum", "size"]
).pairs.rename(
    columns=conn_prob.columns
)
conn_prob.definition(result.loc[0])

In [ ]:
result = pairs_df[
    ["soma_distance"] + ["pairs"]
].assign(group=0
).groupby(
    ["group", "soma_distance"]
).agg(
    ["sum", "size"]
).pairs.rename(
    columns=conn_prob.columns
).assign(
    **{conn_prob.phenomenon: conn_prob.definition}
).loc[0][
    conn_prob_sd.measurement_label
]
result

In [ ]:
result = pairs_df[
    ["pairs", "soma_distance"]
].groupby(
    ["soma_distance"]
).agg(
    ["sum", "size"]
).pairs.rename(
    columns=conn_prob_sd.columns
).assign(**{
    conn_prob_sd.phenomenon: conn_prob_sd.definition})
result[
    conn_prob_sd.phenomenon
].head()

In [ ]:
result = pairs_df[
    ["pairs"]
].agg(
    conn_prob.aggregators
).pairs.rename(
    columns=conn_prob.columns
)
result

In [ ]:
result = pairs_df[
    ["pairs", "soma_distance"]
].groupby(
    ["soma_distance"]
).agg(
    ["sum", "size"]
).pairs.rename(
    columns=conn_prob.columns
).assign(**{
    conn_prob_sd.phenomenon: conn_prob_sd.definition})

result[conn_prob_sd.measurement_label]


In [ ]:
from neuro_dmt.models.bluebrain.circuit.model.pathway import GroupByVariables

result_dataframe_groupby = conn_prob_sd(
    pre_synaptic_cell_group=circuit_model.cells.sample(n=100),
    post_synaptic_cell_group=circuit_model.cells.sample(n=100),
    groupby=GroupByVariables(frozenset({"mtype",}), frozenset({"mtype",})))

In [ ]:
from neuro_dmt.models.bluebrain.circuit.model.cell_type import CellType
result_series = conn_prob_sd(
    pathway=CellType.pathway(
        {"mtype": "L6_UPC"},
        {"mtype": "L6_TPC:A"}))

In [ ]:
result_series.apply(
    lambda x: x + np.random.uniform())

In [ ]:
from dmt.data.observation.measurement.collection import series_type, primitive_type
measured_values = series_type([
    ({"pre_synaptic_mtype": "L23_MC", "post_synaptic_mtype": "L23_MC"},
     result_series.apply(lambda x: x + 0.01 * np.random.uniform())),
    ({"pre_synaptic_mtype": "L23_MC", "post_synaptic_mtype": "L23_MC"},
     result_series.apply(lambda x: x + 0.01 * np.random.uniform())),
    ({"pre_synaptic_mtype": "L23_MC", "post_synaptic_mtype": "L23_MC"},
     result_series.apply(lambda x: x + 0.01 * np.random.uniform())),
    ({"pre_synaptic_mtype": "L23_MC", "post_synaptic_mtype": "L6_TPC:A"},
     result_series.apply(lambda x: x + 0.01 * np.random.uniform())),
    ({"pre_synaptic_mtype": "L23_MC", "post_synaptic_mtype": "L6_TPC:A"},
     result_series.apply(lambda x: x + 0.01 * np.random.uniform())),
    ({"pre_synaptic_mtype": "L23_MC", "post_synaptic_mtype": "L6_TPC:A"},
     result_series.apply(lambda x: x + 0.01 * np.random.uniform())),
    ({"pre_synaptic_mtype": "L6_UPC", "post_synaptic_mtype": "L6_TPC:A"},
     result_series.apply(lambda x: x + 0.01 * np.random.uniform())),
    ({"pre_synaptic_mtype": "L6_UPC", "post_synaptic_mtype": "L6_TPC:A"},
     result_series.apply(lambda x: x + 0.01 * np.random.uniform())),
    ({"pre_synaptic_mtype": "L6_UPC", "post_synaptic_mtype": "L6_TPC:A"},
     result_series.apply(lambda x: x + 0.01 * np.random.uniform()))]
).rename(columns={"value": "connection_probability"})

In [ ]:
from dmt.data.observation import measurement
measured_data = measurement.get_summary(measured_values).reset_index()

In [ ]:
from neuro_dmt.models.bluebrain.circuit.test.test_connectome_analysis import *
phenomenon = Phenomenon(
    "Connection Probability",
    """
    Probability that two neurons in a pathway are connected.
    While mostly interested in `mtype->mtype` pathways, we can define
    a pathway as the group of connected neurons with pre-synaptic and
    post-synaptic neurons belonging to two parametrically defined groups.
    As an example consider neuron populations defined by not only `mtype`
    but also by soma-distance from a given location.
    """,
    group="Connectome")
number_pathways = 5
pathways =\
    mock_circuit_model.pathways(frozenset(("mtype",)))\
        .sample(n=number_pathways)
analysis_test = BlueBrainCircuitAnalysisTest(
    analysis=BrainCircuitAnalysis(
    phenomenon=phenomenon,
    AdapterInterface=ConnectionProbabilityInterface,
    measurement_parameters=Parameters(pathways),
    plotter=HeatMap(
        xvar=("pre_synaptic", "mtype"),
        xlabel="pre-mtype",
        yvar=("post_synaptic", "mtype"),
        ylabel="post-mtype",
        vvar=("connection_probability", "mean"))))

connection_probability_measurement =\
    analysis_test.test_get_measurement(mock_circuit_model, sample_size=10)

analysis = analysis_test._circuit_analysis_test.analysis
adapter = analysis_test.adapter

In [ ]:
m = connection_probability_measurement["BlueBrainCircuitModelMockLight"]
m.groupby(m.index.names).agg(["size", "mean", "std"])

In [ ]:
adapter.get_connection_probability(
    mock_circuit_model,
    pre_synaptic={"mtype": "L23_MC"},
    post_synaptic={"mtype": "L23_MC"})

In [ ]:
def _join(parameters_measurement, value_measurement):
    """..."""
    parameters_measurement = pd.Series(parameters_measurement)
    return\
            pd.Series(
                parameters_measurement)\
              .append(
                  pd.Series({"value": value_measurement}))

In [ ]:
parameter_values = analysis.measurement_parameters.for_sampling(size=2)
measured_values = np.random.uniform(size=len(parameter_values))
pd.concat([
    _join(*args)
    for args in zip(parameter_values, measured_values)])

In [ ]:
analysis.measurement_parameters.join(
    analysis.measurement_parameters.as_dataframe(parameter_values),
    pd.DataFrame({"connection_probability": measured_values}))

In [ ]:
pd.testing.assert_frame_equal(
    measurement.concat_as_samples(
        {"x": measured_values}).loc["x"],
    measured_values)

In [ ]:
from dmt.tk.parameters import index_tree
pd.Series(index_tree.as_unnested_dict(parameter_values[0]))

In [ ]:
plotting_data = measured_values.loc[
    ("L23_MC",)
].reset_index(
).assign(
    soma_distance = lambda df:[np.mean(sd) for sd in df.soma_distance]
)
plotting_data.head()

In [ ]:
import seaborn; seaborn.set
axes = seaborn.lineplot(
    y="connection_probability",
    x="soma_distance",
    hue="post_synaptic_mtype",
    data=plotting_data)

In [ ]:
plotting_data =\
    measured_values.reset_index(
    ).assign(
        soma_distance=lambda df: df.soma_distance.apply(np.mean)
    )
plotting_data.head()

In [ ]:
import matplotlib.pyplot as plt
graphic =\
    seaborn.FacetGrid(
        plotting_data,
        col=None,#"post_synaptic_mtype",
        hue="pre_synaptic_mtype")
graphic.map(plt.hist, "connection_probability")
graphic.add_legend()

In [ ]:
graphic =\
    seaborn.FacetGrid(
        plotting_data,
        hue="pre_synaptic_mtype",
        col="post_synaptic_mtype")
graphic.map(plt.hist, "connection_probability")
graphic.add_legend()

In [ ]:
graphic =\
    seaborn.FacetGrid(
        plotting_data,
        col="post_synaptic_mtype",
        hue="pre_synaptic_mtype")
        #legend_out=True)
graphic.map(
    seaborn.lineplot, 
    "soma_distance",
    "connection_probability",
    alpha=0.7)
graphic.add_legend()

In [ ]:
from dmt.tk.plotting.lines import LinePlot
line_plot = LinePlot(
    xvar="soma_distance",
    xlabel="Soma Distance",
    yvar="connection_probability",
    ylabel="Connection Probability",
    gvar="pre_synaptic_mtype",
    fvar="post_synaptic_mtype")

In [ ]:
line_plot.get_figure({
    "0": measured_values.reset_index().assign(
        soma_distance=lambda df: df.soma_distance.apply(np.mean)
    ).set_index(
        ["pre_synaptic_mtype", "post_synaptic_mtype", "soma_distance"])})

In [ ]:
result = conn_prob_sd(
    pathway=CellType.pathway(
        {"mtype": "L23_MC"},
        {"mtype": "L23_MC"}))
result

In [ ]:
result.reset_index(
).assign(
    soma_distance=lambda df: df.soma_distance.apply(np.mean)
).set_index(
    "soma_distance"
).connection_probability